# Register Client and Create Access Token Notebook
- Find detailed information about client registration and access tokens in this blog post: [Authentication to SAS Viya: a couple of approaches](https://blogs.sas.com/content/sgf/2021/09/24/authentication-to-sas-viya/)
- Additional access token information is found at the end of this notebook.


### Run the cells below and follow the resulting instructions.

# Import packages and create variables

In [1]:
import requests
import json
import os
import base64

# set/create variables
client_id="api.client"     # api.client
client_secret="api.secret" # api.secret
baseURL = "https://ssemonthly.demo.sas.com"     # https://sasserver.sas.com

# Create access token

In [2]:
# create authorization url
codeURL = baseURL + "/SASLogon/oauth/authorize?client_id=" + client_id + "&response_type=code"

# enccode client string
client_string = client_id + ":" + client_secret
message_bytes = client_string.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

# promt with instructions and entry for auth code
print(f"* Please visit the following site {codeURL} in an Incognito browser window")
print("* If provided a login prompt, add your SAS login credentials with SASAdministrator rights")
print("* Once authenticated, you'll be redirected to an authoriztion screen, check all of the boxes that appear")
print("* This will result in a short string of numbers and letters such as `VAxVFVEnKr`; this is your authorization code; copy the code")
code = input("Please enter the authoriztion code you generated through the previous instructions, and then press Enter: ")

# generate API call for access token
url = f"{baseURL}/SASLogon/oauth/token#authorization_code"
payload = "grant_type=authorization_code&code=" + code
headers = {
'Accept': 'application/json',
'Content-Type': 'application/x-www-form-urlencoded',
'Authorization': "Basic " + base64_message
}

# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
access_token = json.loads(response.text)['access_token']
refresh_token = json.loads(response.text)['refresh_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access_token.txt file 
directory = os.getcwd()
with open(directory + '/access_token.txt', 'w') as f:
    f.write(access_token)
print('The access token was stored for you as ' + directory + '/access_token.txt')

# Create refresh_token.txt file 
directory = os.getcwd()
with open(directory + '/refresh_token.txt', 'w') as f:
    f.write(refresh_token)
print('The refresh token was stored for you as ' + directory + '/refresh_token.txt')

* Please visit the following site https://ssemonthly.demo.sas.com/SASLogon/oauth/authorize?client_id=api.client&response_type=code in an Incognito browser window
* If provided a login prompt, add your SAS login credentials with SASAdministrator rights
* Once authenticated, you'll be redirected to an authoriztion screen, check all of the boxes that appear
* This will result in a short string of numbers and letters such as `VAxVFVEnKr`; this is your authorization code; copy the code
Please enter the authoriztion code you generated through the previous instructions, and then press Enter: 7Simtq7mH84hTntrnTzfz3sR2Ht7yLXr


C:\Users\jpnpul\AppData\Local\miniconda3\envs\ml\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ssemonthly.demo.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{
    "access_token": "eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vbG9jYWxob3N0L1NBU0xvZ29uL3Rva2VuX2tleXMiLCJraWQiOiJsZWdhY3ktdG9rZW4ta2V5IiwidHlwIjoiSldUIn0.eyJqdGkiOiJhNzg1NDQ4MGYwNjM0NGU5OWEzNzI4NDc2YzQyYWVhMCIsImV4dF9pZCI6InkwU3BwcXJnM0NkblpzTG5xTTRiWnVQaHBjUnhQVEh1VmtCTzFXMUZjNG8iLCJyZW1vdGVfaXAiOiIxMTMuMzUuMTA1LjM0Iiwic2Vzc2lvbl9zaWciOiJhM2RmNmY5ZS0yZTQ5LTRhMjItYjQ4Ni01MDNmZGYyYzBhMDUiLCJhdXRob3JpdGllcyI6WyJbRUVDXSBEZW1vIENlbnRlciBVc2VycyIsIkRlZmF1bHQgRGVtbyBDZW50ZXIgVXNlcnMiLCJEYXRhQnVpbGRlcnMiLCJTQVNTY29yZVVzZXJzIiwiQ2F0YWxvZy5TdWJqZWN0TWF0dGVyRXhwZXJ0cyIsIkFwcGxpY2F0aW9uQWRtaW5pc3RyYXRvcnMiLCJJQ1VzIiwiTWlncmF0aW9uQWRtaW5zIiwiRXNyaVVzZXJzIl0sInN1YiI6ImNjODg4ZGIzLWY1NDktNDlmMS05OGQzLTczMzg0MzBmMzFlZSIsInNjb3BlIjpbIm9wZW5pZCIsInVhYS51c2VyIl0sImNsaWVudF9pZCI6ImFwaS5jbGllbnQiLCJjaWQiOiJhcGkuY2xpZW50IiwiYXpwIjoiYXBpLmNsaWVudCIsImdyYW50X3R5cGUiOiJhdXRob3JpemF0aW9uX2NvZGUiLCJ1c2VyX2lkIjoiY2M4ODhkYjMtZjU0OS00OWYxLTk4ZDMtNzMzODQzMGYzMWVlIiwib3JpZ2luIjoiYXp1cmUiLCJ1c2VyX25hbWUiOiJQdWx1bmcuV

## Notes on the access token
- The access token has a 12 hour time-to-live (ttl).
- Use the refresh token to generate a new access token.
- The refressh token has a 90 day ttl.
- The access_token is valid in this Notebook and is transferable to other notebooks and used for external API calls.

# Use the refresh token to generate a new access token

In [ ]:
# enccode client string
client_string = client_id + ":" + client_secret
message_bytes = client_string.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

# copy resfresh token from txtfile
directory = os.getcwd()
file = open(directory + "/refresh_token.txt")
# read the file as a list
refresh_token = file.readlines()
# close the file
file.close()
print(refresh_token)
type(refresh_token)

url = f"{baseURL}/SASLogon/oauth/token#refresh_token"

payload = "grant_type=refresh_token&refresh_token=" + ' '.join(refresh_token)
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'Authorization': "Basic " + base64_message
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

# print(response.text)
# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
access_token = json.loads(response.text)['access_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access_token.txt file 
directory = os.getcwd()
with open(directory + '/access_token.txt', 'w') as f:
    f.write(access_token)
print('The access token was stored for you as ' + directory + '/access_token.txt')

# Test connection to SAS Viya via SWAT

In [5]:
import swat
httpconn = swat.CAS("https://ssemonthly.demo.sas.com/cas-shared-default-http", username=None, password=access_token, ssl_ca_list="C:/code/certificate/ssemonthly-rootCA-Intermidiates_4CLI.pem", protocol="https")
httpconn

CAS('ssemonthly.demo.sas.com', 443, protocol='https', name='py-session-1', session='e9d9df2b-7f5f-d44a-8f62-b6f7bc2acd75')

In [6]:
access_token

'eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vbG9jYWxob3N0L1NBU0xvZ29uL3Rva2VuX2tleXMiLCJraWQiOiJsZWdhY3ktdG9rZW4ta2V5IiwidHlwIjoiSldUIn0.eyJqdGkiOiJhNzg1NDQ4MGYwNjM0NGU5OWEzNzI4NDc2YzQyYWVhMCIsImV4dF9pZCI6InkwU3BwcXJnM0NkblpzTG5xTTRiWnVQaHBjUnhQVEh1VmtCTzFXMUZjNG8iLCJyZW1vdGVfaXAiOiIxMTMuMzUuMTA1LjM0Iiwic2Vzc2lvbl9zaWciOiJhM2RmNmY5ZS0yZTQ5LTRhMjItYjQ4Ni01MDNmZGYyYzBhMDUiLCJhdXRob3JpdGllcyI6WyJbRUVDXSBEZW1vIENlbnRlciBVc2VycyIsIkRlZmF1bHQgRGVtbyBDZW50ZXIgVXNlcnMiLCJEYXRhQnVpbGRlcnMiLCJTQVNTY29yZVVzZXJzIiwiQ2F0YWxvZy5TdWJqZWN0TWF0dGVyRXhwZXJ0cyIsIkFwcGxpY2F0aW9uQWRtaW5pc3RyYXRvcnMiLCJJQ1VzIiwiTWlncmF0aW9uQWRtaW5zIiwiRXNyaVVzZXJzIl0sInN1YiI6ImNjODg4ZGIzLWY1NDktNDlmMS05OGQzLTczMzg0MzBmMzFlZSIsInNjb3BlIjpbIm9wZW5pZCIsInVhYS51c2VyIl0sImNsaWVudF9pZCI6ImFwaS5jbGllbnQiLCJjaWQiOiJhcGkuY2xpZW50IiwiYXpwIjoiYXBpLmNsaWVudCIsImdyYW50X3R5cGUiOiJhdXRob3JpemF0aW9uX2NvZGUiLCJ1c2VyX2lkIjoiY2M4ODhkYjMtZjU0OS00OWYxLTk4ZDMtNzMzODQzMGYzMWVlIiwib3JpZ2luIjoiYXp1cmUiLCJ1c2VyX25hbWUiOiJQdWx1bmcuV2Fza2l0b0BzYXMuY29tIiw

In [4]:
import swat
binconn = swat.CAS("20.81.59.109", protocol="cas", username=None, password=access_token, ssl_ca_list="C:/code/certificate/ssemonthly-bin.crt")
binconn

ERROR: Failed to connect to host '20.81.59.109', port 5570.


SWATError: Could not connect to '20.81.59.109' on port 5570.

# Viya Rest call from Python Notebook 

In [ ]:
# Viya call uses hostname
url = "https://ssemonthly.demo.sas.com/reports/reports"

payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()

print(response)

# CAS REST call from Python Notebook

In [ ]:
# CAS call uses ip addr
url= "https://20.237.68.50:443/cas-shared-default-http/cas/sessions"


payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False).json()

print(response)